# Traveling Salesman Problem: Algorithm Comparison

This notebook presents a comparison of various TSP optimization algorithms, evaluating their performance on the Lin105 benchmark dataset.

**Algorithms Analyzed:**
- Random Solver (baseline)
- Nearest Neighbor
- Simulated Annealing (with different initialization strategies)
- Genetic Algorithm (with different initialization strategies)

**Evaluation Metrics:**
- Solution quality (final cost vs optimal)
- Convergence speed (iterations to solution)
- Computational efficiency (steps per second)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from pathlib import Path
from multiprocessing import Pool, cpu_count

from algorithm.nearest_neighbor import NearestNeighbor
from algorithm.simulated_annealing import SimulatedAnnealing
from algorithm.genetic_algo import GeneticAlgorithmSolver
from algorithm.random_solver import RandomSolver
from util import exponential_cooling, find_optimal_tour, run_algorithm_with_timing, run_algorithm_with_iterations

# Configuration
MAX_ITERATIONS = 1_000
MAX_SECONDS = 20.0
RANDOM_SEED = 42
COOLING_RATE = 0.9999
N_RUNS = 10

ALGORITHMS = [
    "Random Solver",
    "Nearest Neighbor",
    "SA (NN-init)",
    "SA (Random-init)",
    "GA (NN-init)",
    "GA (Random-init)"
]

random.seed(RANDOM_SEED)

# Plot styling
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 10

In [ ]:
problem_instance_path = Path("dataset/lin105.tsp")
instance, optimal_cost = find_optimal_tour(problem_instance_path)

In [ ]:
seed_random = list(range(len(instance.cities)))
random.shuffle(seed_random)

nn_builder = NearestNeighbor(instance)
nn_builder.initialize()
for _ in range(len(instance.cities) - 1):
    _ = nn_builder.step()
seed_nn = nn_builder.get_route()

# Algorithm configuration
T0 = 935
exp_schedule = exponential_cooling(COOLING_RATE)

# GA parameters optimized through systematic hyperparameter tuning
GA_POP_SIZE = 30
GA_MUTATION = 0.3
GA_CROSSOVER = 0.6
GA_ELITISM = 1

# Algorithm factory for parallel execution
def create_algorithm_instance(name, instance, run_idx, seed_nn, seed_identity):
    """Factory function to create algorithm instances with proper seeding."""
    T0 = 100
    exp_schedule = exponential_cooling(COOLING_RATE)
    seed = RANDOM_SEED + run_idx
    
    algorithm_configs = {
        "Random Solver": (RandomSolver(instance, seed=seed), None),
        "Nearest Neighbor": (NearestNeighbor(instance), None),
        "SA (NN-init)": (SimulatedAnnealing(instance, T0, exp_schedule, seed=seed), seed_nn),
        "SA (Random-init)": (SimulatedAnnealing(instance, T0, exp_schedule, seed=seed), None),
        "GA (NN-init)": (
            GeneticAlgorithmSolver(instance, seed=seed, population_size=GA_POP_SIZE,
                                 mutation_rate=GA_MUTATION, crossover_rate=GA_CROSSOVER,
                                 num_parents=2, num_child=2, elitism_count=GA_ELITISM),
            seed_nn
        ),
        "GA (Random-init)": (
            GeneticAlgorithmSolver(instance, seed=seed, population_size=GA_POP_SIZE,
                                 mutation_rate=GA_MUTATION, crossover_rate=GA_CROSSOVER,
                                 num_parents=2, num_child=2, elitism_count=GA_ELITISM),
            None
        ),
    }
    
    return algorithm_configs[name]

# Helper function for parallel execution
def run_single_trial(args):
    """Run a single algorithm trial - used for parallel processing."""
    name, run_idx, instance_data, seed_nn_data, seed_identity_data = args
    
    # Recreate instance from data (needed for multiprocessing)
    from tsp.model import TSPInstance
    instance = TSPInstance(name=instance_data['name'], cities=instance_data['cities'])
    
    solver, init_route = create_algorithm_instance(name, instance, run_idx, seed_nn_data, seed_identity_data)
    
    iterations, best_costs, current_costs, times, best_route = run_algorithm_with_timing(
        instance, solver, init_route, MAX_SECONDS
    )
    
    return {
        'iterations': iterations,
        'best_costs': best_costs,
        'current_costs': current_costs,
        'times': times,
        'best_route': best_route
    }

# Time-based benchmark with multiple runs (parallelized)
print("=" * 70)
print(f"TIME-BASED BENCHMARK ({MAX_SECONDS}s per algorithm, {N_RUNS} runs, {cpu_count()} CPUs)")
print("=" * 70)

# Prepare instance data for serialization
instance_data = {'name': instance.name, 'cities': instance.cities}
seed_nn_data = seed_nn

time_results = {}
for name in ALGORITHMS:
    print(f"Running {name}... ({N_RUNS} runs in parallel)", end=" ", flush=True)
    
    # Prepare arguments for parallel execution
    args_list = [(name, run_idx, instance_data, seed_nn_data, seed_random) 
                 for run_idx in range(N_RUNS)]
    
    # Run in parallel
    with Pool(processes=min(cpu_count(), N_RUNS)) as pool:
        all_runs = pool.map(run_single_trial, args_list)
    
    # Align all runs to common time grid using interpolation
    # Find the run that reached closest to MAX_SECONDS
    max_time_reached = max(run['times'][-1] for run in all_runs)
    
    # Create uniform time grid from 0 to the maximum time reached
    num_points = 100  # Use 100 points for smooth curves
    common_times = np.linspace(0, min(max_time_reached, MAX_SECONDS), num_points)
    
    # Interpolate each run onto common time grid
    aligned_best = []
    for run in all_runs:
        # Interpolate best costs onto common time grid
        interp_best = np.interp(common_times, run['times'], run['best_costs'])
        aligned_best.append(interp_best)
    
    aligned_best = np.array(aligned_best)
    aligned_times = common_times
    
    # Calculate statistics
    mean_best = np.mean(aligned_best, axis=0)
    std_best = np.std(aligned_best, axis=0)
    min_best = np.min(aligned_best, axis=0)
    max_best = np.max(aligned_best, axis=0)
    
    final_costs = [run['best_costs'][-1] for run in all_runs]
    
    time_results[name] = {
        'times': aligned_times,
        'mean_best': mean_best,
        'std_best': std_best,
        'min_best': min_best,
        'max_best': max_best,
        'all_runs_best': aligned_best,
        'final_cost_mean': np.mean(final_costs),
        'final_cost_std': np.std(final_costs),
        'final_cost_min': np.min(final_costs),
        'final_cost_max': np.max(final_costs),
        'total_iterations': np.mean([len(run['iterations']) for run in all_runs])
    }
    
    print(f"Mean: {time_results[name]['final_cost_mean']:.2f} ± {time_results[name]['final_cost_std']:.2f}")


In [ ]:
# Helper function for iteration-based parallel execution
def run_single_iteration_trial(args):
    """Run a single iteration-based trial."""
    name, run_idx, instance_data, seed_nn_data, seed_identity_data = args
    
    from tsp.model import TSPInstance
    instance = TSPInstance(name=instance_data['name'], cities=instance_data['cities'])
    
    solver, init_route = create_algorithm_instance(name, instance, run_idx, seed_nn_data, seed_identity_data)
    
    iters, best_costs, current_costs, times, best_route = run_algorithm_with_iterations(
        instance, solver, init_route, MAX_ITERATIONS
    )
    
    return {'iters': iters, 'best_costs': best_costs, 'times': times, 'best_route': best_route}

# Iteration-based benchmark with multiple runs (parallelized)
print("\n" + "=" * 70)
print(f"ITERATION-BASED BENCHMARK ({MAX_ITERATIONS} iterations, {N_RUNS} runs)")
print("=" * 70)

iteration_stats = {}
for name in time_results.keys():
    print(f"Running {name}... ({N_RUNS} runs in parallel)", end=" ", flush=True)
    
    args_list = [(name, run_idx, instance_data, seed_nn_data, seed_random) 
                 for run_idx in range(N_RUNS)]
    
    with Pool(processes=min(cpu_count(), N_RUNS)) as pool:
        results = pool.map(run_single_iteration_trial, args_list)
    
    runs_best = [r['best_costs'] for r in results]
    runs_iters = [r['iters'] for r in results]
    runs_time = [r['times'] for r in results]
    
    # Align by min length
    min_len = min(len(x) for x in runs_best)
    aligned_best = [run[:min_len] for run in runs_best]
    aligned_iters = runs_iters[0][:min_len]
    
    mean_best = np.mean(aligned_best, axis=0)
    std_best = np.std(aligned_best, axis=0)
    final_costs = [run[-1] for run in runs_best if run]
    total_times = [t[-1] for t in runs_time if t]
    
    iteration_stats[name] = {
        'iterations': aligned_iters,
        'mean_best': mean_best,
        'std_best': std_best,
        'final_cost_mean': np.mean(final_costs) if final_costs else float('inf'),
        'final_cost_std': np.std(final_costs) if final_costs else 0.0,
        'total_time_mean': np.mean(total_times) if total_times else 0.0,
    }
    
    print(f"Cost: {iteration_stats[name]['final_cost_mean']:.2f} ± {iteration_stats[name]['final_cost_std']:.2f}")

In [ ]:
# Summary Table
print("\n" + "=" * 85)
print("SUMMARY")
print("=" * 85)
print(f"{'Algorithm':<20} {'Mean Cost':<18} {'vs Optimal':<12} {'Steps/sec':<12} {'CV %':<8}")
print("-" * 85)

for name in time_results.keys():
    mean_cost = time_results[name]['final_cost_mean']
    std_cost = time_results[name]['final_cost_std']
    cost_str = f"{mean_cost:.1f} ± {std_cost:.1f}"
    vs_optimal = f"+{((mean_cost / optimal_cost - 1) * 100):.1f}%" if optimal_cost else "N/A"
    steps_per_sec = time_results[name]['total_iterations']
    cv = (std_cost / mean_cost * 100)
    print(f"{name:<20} {cost_str:<18} {vs_optimal:<12} {steps_per_sec:<12.0f} {cv:<8.1f}")

print(f"{'Optimal':<20} {optimal_cost:<18.2f} {'0.0%':<12} {'-':<12} {'-':<8}")
print("=" * 85)


In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

colors = plt.cm.tab10(np.linspace(0, 1, len(time_results)))
for (name, data), color in zip(time_results.items(), colors):
    # Plot mean line
    ax.plot(data['times'], data['mean_best'], label=name, linewidth=2.5, color=color, alpha=0.9)
    
    # Add shaded region showing min-max bounds across runs
    ax.fill_between(data['times'], data['min_best'], data['max_best'], 
                    color=color, alpha=0.15, linewidth=0)

if optimal_cost:
    ax.axhline(y=optimal_cost, color='darkgreen', linestyle='--', linewidth=2, alpha=0.7, label='Optimal Solution')

ax.set_xlabel('Time (seconds)', fontsize=13, fontweight='bold')
ax.set_ylabel('Best Cost Found (mean, shaded: min-max)', fontsize=13, fontweight='bold')
ax.set_title(f'Algorithm Convergence Over Time ({N_RUNS} runs) - {instance.name.upper()}', 
             fontsize=15, fontweight='bold', pad=20)
ax.legend(loc='upper right', framealpha=0.9, fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# --- Solution Quality Comparison Plot ---
fig1, ax1 = plt.subplots(figsize=(8, 7))

names = list(time_results.keys())
final_costs_mean = [time_results[name]['final_cost_mean'] for name in names]
final_costs_std = [time_results[name]['final_cost_std'] for name in names]

colors = plt.cm.tab10(np.linspace(0, 1, len(names)))
bars1 = ax1.bar(range(len(names)), final_costs_mean, yerr=final_costs_std, 
                capsize=5, color=colors, alpha=0.8, edgecolor='black')

if optimal_cost:
    ax1.axhline(y=optimal_cost, color='darkgreen', linestyle='--', linewidth=2, alpha=0.7, label='Optimal')
    ax1.legend()

ax1.set_xticks(range(len(names)))
ax1.set_xticklabels(names, rotation=45, ha='right')
ax1.set_ylabel('Final Cost (mean ± std)', fontsize=12, fontweight='bold')
ax1.set_title(f'Solution Quality Comparison ({N_RUNS} runs)', fontsize=14, fontweight='bold', pad=15)
ax1.grid(True, alpha=0.3, axis='y', linestyle='--')

for i, (cost, err) in enumerate(zip(final_costs_mean, final_costs_std)):
    ax1.text(i, cost + err + max(final_costs_mean) * 0.02, f'{cost:.0f}', 
             ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

# --- Computational Efficiency Plot ---
fig2, ax2 = plt.subplots(figsize=(8, 7))

steps_per_sec = [time_results[name]['total_iterations'] / MAX_SECONDS for name in names]
bars2 = ax2.bar(range(len(names)), steps_per_sec, color=colors, alpha=0.8, edgecolor='black')

ax2.set_xticks(range(len(names)))
ax2.set_xticklabels(names, rotation=45, ha='right')
ax2.set_ylabel('Steps per Second', fontsize=12, fontweight='bold')
ax2.set_title('Computational Efficiency', fontsize=14, fontweight='bold', pad=15)
ax2.grid(True, alpha=0.3, axis='y', linestyle='--')

for i, rate in enumerate(steps_per_sec):
    ax2.text(i, rate + max(steps_per_sec) * 0.02, f'{rate:.0f}', 
             ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Performance Analysis Summary
print("\nKEY FINDINGS:\n")
print("1. SOLUTION QUALITY:")

best_algorithm = min(time_results.items(), key=lambda x: x[1]['final_cost_mean'])
print(f"   - Best performer: {best_algorithm[0]}")
print(f"   - Final cost: {best_algorithm[1]['final_cost_mean']:.2f} ± {best_algorithm[1]['final_cost_std']:.2f}")
if optimal_cost:
    gap = ((best_algorithm[1]['final_cost_mean'] / optimal_cost - 1) * 100)
    print(f"   - Optimality gap: {gap:.1f}%")
print(f"   - Best run achieved: {best_algorithm[1]['final_cost_min']:.2f}")
print(f"   - Worst run: {best_algorithm[1]['final_cost_max']:.2f}")

print("\n2. COMPUTATIONAL EFFICIENCY:")
fastest = max(time_results.items(), key=lambda x: x[1]['total_iterations'])
print(f"   - Fastest algorithm: {fastest[0]}")
print(f"   - Steps per second: {fastest[1]['total_iterations']:.0f}")

print("\n3. CONVERGENCE SPEED:")
for name in ['SA (NN-init)', 'GA (NN-init)', 'Nearest Neighbor']:
    if name in time_results:
        data = time_results[name]
        if len(data['mean_best']) > 10:
            # Check how quickly it gets close to final solution
            final = data['mean_best'][-1]
            threshold = final * 1.1  # Within 10% of final
            converged_idx = next((i for i, cost in enumerate(data['mean_best']) if cost <= threshold), len(data['mean_best']))
            converge_time = data['times'][converged_idx] if converged_idx < len(data['times']) else data['times'][-1]
            print(f"   - {name}: converged in {converge_time:.3f}s (average)")

print(f"\n4. VARIABILITY ACROSS RUNS:")
for name in time_results.keys():
    variance = time_results[name]['final_cost_std'] / time_results[name]['final_cost_mean'] * 100
    print(f"   - {name}: CV = {variance:.1f}%")

## Conclusions

Based on comprehensive benchmarking on Lin105 (10 runs per algorithm, 5s runtime):

Both SA variants achieve ~9% optimality gap with high throughput (136k–146k steps/sec) and low variability (CV ~3%). Random initialization performs slightly better than NN-init, suggesting NN seeding less critical for SA on larger instances.

GA (NN-init) achieves only 18% gap despite NN seeding. GA (Random-init) fails to beat Nearest Neighbor baseline and shows high instability (CV 9.4%). Both GA variants are 100× slower than SA due to population overhead.

Initialization affects GA highly (NN-init reduces gap from 58% to 18%) but minimal for SA (both variants ~9%) in this example.